# Mekanisme Perhatian dan Transformer

Satu kelemahan utama rangkaian berulang (recurrent networks) ialah semua perkataan dalam satu urutan mempunyai kesan yang sama terhadap hasilnya. Ini menyebabkan prestasi yang kurang optimum dengan model encoder-decoder LSTM standard untuk tugas urutan ke urutan, seperti Pengecaman Entiti Bernama (Named Entity Recognition) dan Terjemahan Mesin. Dalam realiti, perkataan tertentu dalam urutan input selalunya mempunyai lebih banyak kesan terhadap output berurutan berbanding yang lain.

Pertimbangkan model urutan ke urutan, seperti terjemahan mesin. Ia dilaksanakan oleh dua rangkaian berulang, di mana satu rangkaian (**encoder**) akan memampatkan urutan input ke dalam keadaan tersembunyi, dan satu lagi, **decoder**, akan mengembangkan keadaan tersembunyi ini menjadi hasil terjemahan. Masalah dengan pendekatan ini ialah keadaan akhir rangkaian akan menghadapi kesukaran untuk mengingati permulaan ayat, menyebabkan kualiti model yang rendah pada ayat yang panjang.

**Mekanisme Perhatian** menyediakan cara untuk memberi pemberat kepada kesan kontekstual setiap vektor input terhadap setiap ramalan output RNN. Cara ia dilaksanakan adalah dengan mencipta pintasan antara keadaan perantaraan RNN input dan RNN output. Dengan cara ini, apabila menghasilkan simbol output $y_t$, kita akan mengambil kira semua keadaan tersembunyi input $h_i$, dengan pekali pemberat yang berbeza $\alpha_{t,i}$.

![Imej menunjukkan model encoder/decoder dengan lapisan perhatian tambahan](../../../../../translated_images/ms/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder dengan mekanisme perhatian tambahan dalam [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), dipetik daripada [catatan blog ini](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matriks perhatian $\{\alpha_{i,j}\}$ akan mewakili tahap di mana perkataan input tertentu memainkan peranan dalam penjanaan perkataan tertentu dalam urutan output. Di bawah adalah contoh matriks seperti itu:

![Imej menunjukkan penjajaran sampel yang ditemui oleh RNNsearch-50, diambil daripada Bahdanau - arviz.org](../../../../../translated_images/ms/bahdanau-fig3.09ba2d37f202a6af.webp)

*Rajah diambil daripada [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Rajah 3)*

Mekanisme perhatian bertanggungjawab untuk sebahagian besar keadaan seni semasa atau hampir semasa dalam Pemprosesan Bahasa Semula Jadi. Walau bagaimanapun, menambah perhatian secara signifikan meningkatkan bilangan parameter model yang membawa kepada isu penskalaan dengan RNN. Satu kekangan utama penskalaan RNN ialah sifat berulang model menjadikannya mencabar untuk melaksanakan latihan secara batch dan selari. Dalam RNN, setiap elemen dalam satu urutan perlu diproses secara berurutan, yang bermaksud ia tidak boleh dengan mudah diparalelkan.

Penggunaan mekanisme perhatian yang digabungkan dengan kekangan ini membawa kepada penciptaan Model Transformer yang kini menjadi Keadaan Seni (State of the Art) yang kita kenal dan gunakan hari ini, daripada BERT hingga OpenGPT3.

## Model Transformer

Daripada meneruskan konteks setiap ramalan sebelumnya ke langkah penilaian seterusnya, **model transformer** menggunakan **pengekodan kedudukan** dan perhatian untuk menangkap konteks input tertentu dalam tetingkap teks yang disediakan. Imej di bawah menunjukkan bagaimana pengekodan kedudukan dengan perhatian dapat menangkap konteks dalam tetingkap tertentu.

![GIF animasi menunjukkan bagaimana penilaian dilakukan dalam model transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Oleh kerana setiap kedudukan input dipetakan secara bebas ke setiap kedudukan output, transformer dapat diparalelkan dengan lebih baik berbanding RNN, yang membolehkan model bahasa yang lebih besar dan lebih ekspresif. Setiap kepala perhatian boleh digunakan untuk mempelajari hubungan yang berbeza antara perkataan yang meningkatkan tugas Pemprosesan Bahasa Semula Jadi hiliran.

**BERT** (Bidirectional Encoder Representations from Transformers) ialah rangkaian transformer berlapis besar dengan 12 lapisan untuk *BERT-base*, dan 24 untuk *BERT-large*. Model ini mula-mula dilatih awal pada korpus teks yang besar (WikiPedia + buku) menggunakan latihan tanpa pengawasan (meramalkan perkataan yang disembunyikan dalam satu ayat). Semasa latihan awal, model menyerap tahap pemahaman bahasa yang signifikan yang kemudiannya boleh dimanfaatkan dengan set data lain menggunakan penalaan halus. Proses ini dipanggil **pembelajaran pemindahan**.

![gambar dari http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ms/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Terdapat banyak variasi seni bina Transformer termasuk BERT, DistilBERT, BigBird, OpenGPT3 dan banyak lagi yang boleh ditala halus. Pakej [HuggingFace](https://github.com/huggingface/) menyediakan repositori untuk melatih banyak seni bina ini dengan PyTorch.

## Menggunakan BERT untuk pengelasan teks

Mari kita lihat bagaimana kita boleh menggunakan model BERT yang telah dilatih awal untuk menyelesaikan tugas tradisional kita: pengelasan urutan. Kita akan mengelaskan set data AG News asal kita.

Pertama, mari kita muatkan perpustakaan HuggingFace dan set data kita:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Oleh kerana kita akan menggunakan model BERT yang telah dilatih, kita perlu menggunakan penanda token tertentu. Pertama, kita akan memuatkan penanda token yang berkaitan dengan model BERT yang telah dilatih.

Perpustakaan HuggingFace mengandungi repositori model yang telah dilatih, yang boleh anda gunakan hanya dengan menentukan nama mereka sebagai argumen kepada fungsi `from_pretrained`. Semua fail binari yang diperlukan untuk model akan dimuat turun secara automatik.

Namun, pada masa tertentu anda mungkin perlu memuatkan model anda sendiri, di mana anda boleh menentukan direktori yang mengandungi semua fail yang berkaitan, termasuk parameter untuk penanda token, fail `config.json` dengan parameter model, berat binari, dan sebagainya.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objek `tokenizer` mengandungi fungsi `encode` yang boleh digunakan secara langsung untuk mengekod teks:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Kemudian, mari kita buat iterator yang akan kita gunakan semasa latihan untuk mengakses data. Oleh kerana BERT menggunakan fungsi pengekodannya sendiri, kita perlu menentukan fungsi padding yang serupa dengan `padify` yang telah kita tentukan sebelum ini:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Dalam kes kita, kita akan menggunakan model BERT yang telah dilatih terlebih dahulu yang dipanggil `bert-base-uncased`. Mari kita muatkan model menggunakan pakej `BertForSequenceClassfication`. Ini memastikan bahawa model kita sudah mempunyai seni bina yang diperlukan untuk klasifikasi, termasuk pengklasifikasi akhir. Anda akan melihat mesej amaran yang menyatakan bahawa berat pengklasifikasi akhir tidak diinisialisasi, dan model memerlukan latihan awal - itu adalah perkara yang sangat baik, kerana itulah yang akan kita lakukan!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Sekarang kita bersedia untuk memulakan latihan! Oleh kerana BERT sudah dilatih terlebih dahulu, kita ingin memulakan dengan kadar pembelajaran yang agak kecil supaya tidak merosakkan berat awal.

Semua kerja keras dilakukan oleh model `BertForSequenceClassification`. Apabila kita memanggil model pada data latihan, ia mengembalikan kedua-dua kehilangan (loss) dan output rangkaian untuk minibatch input. Kita menggunakan kehilangan untuk pengoptimuman parameter (`loss.backward()` melakukan laluan ke belakang), dan `out` untuk mengira ketepatan latihan dengan membandingkan label yang diperoleh `labs` (dikira menggunakan `argmax`) dengan `labels` yang dijangkakan.

Untuk mengawal proses, kita mengumpul kehilangan dan ketepatan sepanjang beberapa iterasi, dan mencetaknya setiap kitaran latihan `report_freq`.

Latihan ini mungkin akan mengambil masa yang agak lama, jadi kita menghadkan bilangan iterasi.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Anda boleh lihat (terutamanya jika anda meningkatkan bilangan iterasi dan menunggu cukup lama) bahawa klasifikasi BERT memberikan ketepatan yang cukup baik! Ini kerana BERT sudah memahami struktur bahasa dengan baik, dan kita hanya perlu melaras pengklasifikasi akhir. Walau bagaimanapun, kerana BERT adalah model yang besar, keseluruhan proses latihan mengambil masa yang lama dan memerlukan kuasa pengkomputeran yang tinggi! (GPU, dan sebaiknya lebih daripada satu).

> **Note:** Dalam contoh kita, kita telah menggunakan salah satu model BERT pra-latih yang paling kecil. Terdapat model yang lebih besar yang berkemungkinan memberikan hasil yang lebih baik.


## Menilai prestasi model

Sekarang kita boleh menilai prestasi model kita pada set data ujian. Gelung penilaian adalah hampir serupa dengan gelung latihan, tetapi kita tidak boleh lupa untuk menukar model kepada mod penilaian dengan memanggil `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Perkara Penting

Dalam unit ini, kita telah melihat betapa mudahnya mengambil model bahasa yang telah dilatih dari perpustakaan **transformers** dan menyesuaikannya untuk tugas klasifikasi teks kita. Begitu juga, model BERT boleh digunakan untuk pengekstrakan entiti, menjawab soalan, dan tugas NLP lain.

Model transformer mewakili teknologi terkini dalam NLP, dan dalam kebanyakan kes, ia seharusnya menjadi penyelesaian pertama yang anda cuba apabila melaksanakan penyelesaian NLP tersuai. Walau bagaimanapun, memahami prinsip asas rangkaian neural berulang yang dibincangkan dalam modul ini adalah sangat penting jika anda ingin membina model neural yang lebih maju.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
